In [5]:
import pandas as pd
import numpy as np
import sklearn
from urllib.request import urlopen
from json import loads
# pd.options.display.float_format = '{:.0f}'.format

In [11]:
twenty_eleven_df = pd.read_csv('APEducationData/schools_crdc_ap_exams_2011.csv', low_memory=False)
twenty_thirteen_df = pd.read_csv('APEducationData/schools_crdc_ap_exams_2013.csv', low_memory=False)
twenty_fifteen_df = pd.read_csv('APEducationData/schools_crdc_ap_exams_2015.csv', low_memory=False)

# 2017's data has a lot of missing values and negative values. We don't consider using it at this moment
# twenty_seventeen_df = pd.read_csv('schools_crdc_ap_exams_2017.csv', low_memory=False)

In [13]:
# Read Enrollment Data
twenty_eleven_enroll_df = pd.read_csv("APEducationData/schools_crdc_enrollment_k12_2011.csv", low_memory=False)
twenty_thirteen_enroll_df = pd.read_csv("APEducationData/schools_crdc_enrollment_k12_2013.csv", low_memory=False)
twenty_fifteen_enroll_df = pd.read_csv("APEducationData/schools_crdc_enrollment_k12_2015.csv", low_memory=False)



In [145]:
# weird_twenty_eleven_df = twenty_eleven_df[
#     ((twenty_eleven_df['students_AP_exam_oneormore'] > twenty_eleven_df['students_AP_pass_oneormore']) & (twenty_eleven_df['students_AP_pass_oneormore'] >= 0)) | 
#     ((twenty_eleven_df['students_AP_exam_all'] > twenty_eleven_df['students_AP_exam_oneormore']) & (twenty_eleven_df['students_AP_exam_oneormore'] >= 0)) |
#     ((twenty_eleven_df['students_AP_pass_all'] > twenty_eleven_df['students_AP_exam_all']) & (twenty_eleven_df['students_AP_exam_all'] >= 0))
                                                                                                                        
# ]
# number_schools = weird_twenty_eleven_df['crdc_id'].nunique()
# print(len(weird_twenty_eleven_df))
# print(number_schools)

In [95]:
# missing_id = twenty_eleven_df[twenty_eleven_df['ncessch'].isna()]
# print(len(missing_id) / 30)

# Function to process data that removes all NA rows or with negative values

def data_validifier(dataframe, state_id):
    # Getting rows with the required state_id
    state_df = dataframe[dataframe['fips'] == state_id];
    
    # Aggregate Race and Sex
    state_agg = state_df[(state_df['race'] == 99) & (state_df['sex'] == 99)]
    
    # Drop some columns we don't need at this moment
    state_df_dropcolumn = state_agg.drop(columns=['students_AP_exam_all', 'students_AP_pass_all', 'fips', 'lep', 'disability', 'leaid', 'crdc_id', 
                                                'students_AP_exam_none', 'students_AP_pass_none', 'students_AP_exam_oneormore', 'sex', 'race'])
    
    # Drop NA Values(It's fine to have NA values for exam_all or pass_all
    state_df_nona = state_df_dropcolumn.dropna()
    
    # Remove all rows with negative values
    state_df_final = state_df_nona[(state_df_nona['students_AP_pass_oneormore'] >= 0)]
    
    return state_df_final

# Get Arizona's valid df
arizona_2011 = data_validifier(twenty_eleven_df, 4)
arizona_2013 = data_validifier(twenty_thirteen_df, 4)
arizona_2015 = data_validifier(twenty_fifteen_df, 4)

print(arizona_2011)


        year  students_AP_pass_oneormore       ncessch
57839   2011                       129.0  4.000010e+10
58379   2011                        11.0  4.000190e+10
58709   2011                         4.0  4.000260e+10
58979   2011                         4.0  4.000430e+10
59669   2011                         4.0  4.000740e+10
...      ...                         ...           ...
115049  2011                         8.0  4.096300e+10
115079  2011                        56.0  4.096300e+10
115109  2011                        15.0  4.096300e+10
115139  2011                        86.0  4.096300e+10
115199  2011                         4.0  4.097330e+10

[126 rows x 3 columns]


In [96]:
# Function to process enrollment data
def enrollment_processor(dataframe, state_id):
    # Getting rows with the relevant state_id
    state_enroll_df = dataframe[dataframe['fips'] == state_id]
    
    # Drop preenrollment 
    state_enroll_df = state_enroll_df.drop(columns=['psenrollment_crdc', 'crdc_id', 'fips'])
    
    # We need proportion of females and each races
    # We don't care about disability or LEP here
    state_enroll_relevant = state_enroll_df[(state_enroll_df['disability'] == 99) & (state_enroll_df['lep'] == 99)]
    state_enroll_relevant = state_enroll_relevant.drop(columns=['disability', 'lep'])
    
    # We only want data with ncessch id
    state_enroll_withid = state_enroll_relevant[~state_enroll_relevant['ncessch'].isna()]
    
    # We want proportion and check that with 99
    state_enroll_final = state_enroll_withid[(state_enroll_withid['race'] == 99) | (state_enroll_withid['sex'] == 99)]
    
    return state_enroll_final


arizona_enroll_2011 = enrollment_processor(twenty_eleven_enroll_df, 4)
arizona_enroll_2013 = enrollment_processor(twenty_thirteen_enroll_df, 4)
arizona_enroll_2015 = enrollment_processor(twenty_fifteen_enroll_df, 4)
print(len(arkansas_enroll_2015))
# We only care about schools that have AP passing one or more in previous table
arizona_enroll_2011 = arizona_enroll_2011[arizona_enroll_2011['ncessch'].isin(arizona_2011['ncessch'])]
arizona_enroll_2013 = arizona_enroll_2013[arizona_enroll_2013['ncessch'].isin(arizona_2013['ncessch'])]
arizona_enroll_2015 = arizona_enroll_2015[arizona_enroll_2015['ncessch'].isin(arizona_2015['ncessch'])]

print(len(arizona_enroll_2011))

1660
1250


In [105]:
# Find the percentage of females and different races
def find_percentage(dataframe):
    total_enroll = dataframe[(dataframe['race'] == 99) & (dataframe['sex'] == 99)].copy()
    total_enroll = total_enroll[['ncessch', 'enrollment_crdc']]
    total_enroll.rename(columns={'enrollment_crdc' : 'total_enrollment'}, inplace=True)
    
    merged_total_df = pd.merge(total_enroll, dataframe)
    
    # Find percentage of female and merge
    female_df = merged_total_df[(merged_total_df['sex'] == 2) & (merged_total_df['race'] == 99)].copy()
    female_df['% female'] = female_df['enrollment_crdc'] / female_df['total_enrollment']
    female_df = female_df[['ncessch', '% female']]
    
    
    white_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 1)].copy()
    white_df['% white'] = white_df['enrollment_crdc'] / white_df['total_enrollment']
    white_df = white_df[['ncessch', '% white']]
    
    black_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 2)].copy()
    black_df['% black'] = black_df['enrollment_crdc'] / black_df['total_enrollment']
    black_df = black_df[['ncessch', '% black']]
    
    hispanic_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 3)].copy()
    hispanic_df['% hispanic'] = hispanic_df['enrollment_crdc'] / hispanic_df['total_enrollment']
    hispanic_df = hispanic_df[['ncessch', '% hispanic']]
    
    asian_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 4)].copy()
    asian_df['% asian'] = asian_df['enrollment_crdc'] / asian_df['total_enrollment']
    asian_df = asian_df[['ncessch', '% asian']]
    
    american_indian_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 5)].copy()
    american_indian_df['% american_indian'] = american_indian_df['enrollment_crdc'] / american_indian_df['total_enrollment']
    american_indian_df = american_indian_df[['ncessch', '% american_indian']]
    
    native_hawaiian_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 6)].copy()
    native_hawaiian_df['% native_hawaiian'] = native_hawaiian_df['enrollment_crdc'] / native_hawaiian_df['total_enrollment']
    native_hawaiian_df = native_hawaiian_df[['ncessch', '% native_hawaiian']]
    
    twoormore_races_df = merged_total_df[(merged_total_df['sex'] == 99) & (merged_total_df['race'] == 7)].copy()
    twoormore_races_df['% two_or_more_races'] = twoormore_races_df['enrollment_crdc'] / twoormore_races_df['total_enrollment']
    twoormore_races_df = twoormore_races_df[['ncessch', '% two_or_more_races']]
    
    merge_female_df = pd.merge(dataframe, female_df, on='ncessch')
    merge_white_df = pd.merge(merge_female_df, white_df, on='ncessch')
    merge_black_df = pd.merge(merge_white_df, black_df, on='ncessch')
    merge_hispanic_df = pd.merge(merge_black_df, hispanic_df, on='ncessch')
    merge_asian_df = pd.merge(merge_hispanic_df, asian_df, on='ncessch')
    merge_american_indian_df = pd.merge(merge_asian_df, american_indian_df, on='ncessch')
    merge_native_hawaiian_df = pd.merge(merge_american_indian_df, native_hawaiian_df, on='ncessch')
    merge_twoormorerace_df = pd.merge(merge_native_hawaiian_df, twoormore_races_df, on='ncessch')
    final_df = pd.merge(merge_twoormorerace_df, total_enroll, on='ncessch')
    
    final_df = final_df[(final_df['race'] == 99) & (final_df['sex'] == 99)]
    final_df = final_df.drop(columns=['race', 'sex', 'enrollment_crdc'])
    return final_df

def get_info_df(percentage_df, score_df):
    
    info_df = pd.merge(percentage_df, score_df, on='ncessch')
    year_col = info_df['year_y']
    info_df = info_df.drop(columns=['year_x', 'year_y'])
    info_df.insert(1, 'year', year_col)
    return info_df

arizona_2011_enroll_percent = find_percentage(arizona_enroll_2011)
arizona_2013_enroll_percent = find_percentage(arizona_enroll_2011)
arizona_2011_info_df = get_info_df(find_percentage(arizona_enroll_2011), arizona_2011)
arizona_2013_info_df = get_info_df(find_percentage(arizona_enroll_2013), arizona_2013)
arizona_2015_info_df = get_info_df(find_percentage(arizona_enroll_2015), arizona_2015)
print(arizona_2011_info_df['ncessch'].nunique())
print(arizona_2011_info_df['leaid'].nunique())

125
54


In [131]:
# Concat Data for training purpose
# Planning to use 2011 and 2013 for training

train_data = pd.concat([california_2011, california_2013], ignore_index = True)

# Drop the year column 
# Reset Index for test data
# train_data = train_data.drop(columns=['year'])
# test_data = california_2015.drop(columns=['year']).reset_index(drop=True)


train_feature = train_data.drop(columns=['students_AP_pass_none', 'students_AP_exam_none', 'students_AP_exam_oneormore', 'students_AP_pass_oneormore'])
train_none_ap = train_data['students_AP_exam_none']
train_none_passing = train_data['students_AP_pass_none']
train_oneormore_ap = train_data['students_AP_exam_oneormore']
train_oneormore_passing = train_data['students_AP_pass_oneormore']

test_feature = test_data.drop(columns=['students_AP_pass_none', 'students_AP_exam_none', 'students_AP_exam_oneormore', 'students_AP_pass_oneormore'])
test_none_ap = test_data['students_AP_exam_none']
test_none_passing = test_data['students_AP_pass_none']
test_oneormore_ap = test_data['students_AP_exam_oneormore']
test_oneormore_passing = test_data['students_AP_pass_oneormore']



,sex,race,ncessch
0,1,1.0,6.000010e+10
1,1,3.0,6.000010e+10
2,1,99.0,6.000010e+10
3,2,1.0,6.000010e+10
4,2,99.0,6.000010e+10
...,...,...,...
95,2,1.0,6.000201e+10
96,2,4.0,6.000201e+10
97,2,99.0,6.000201e+10
98,99,1.0,6.000201e+10


In [134]:
# Standard Scaling our features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(train_feature)
X_test = scaler.transform(test_feature)


In [135]:
# Use RandomForestRegressor to fit the model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
model = RandomForestRegressor(random_state = 482)

# Function to fit and predict different metrics
def predictor():
    
    # Fit and predict students_AP_exam_none
    model.fit(X_train, train_none_ap)
    predict_none_ap = model.predict(X_test)
    
    # Fit and predict students_AP_pass_none    
    model.fit(X_train, train_none_passing)
    predict_none_passing = model.predict(X_test)
    
    # Fit and predict students_AP_exam_oneormore
    model.fit(X_train, train_oneormore_ap)
    predict_oneormore_ap = model.predict(X_test)
    
    # Fit and predict students_AP_pass_oneormore
    model.fit(X_train, train_oneormore_passing)
    predict_oneormore_passing = model.predict(X_test)
    
    # Calculate the MSE metrics for each
    none_ap_mse = mean_squared_error(predict_none_ap, test_none_ap)
    none_passing_mse = mean_squared_error(predict_none_passing, test_none_passing)
    oneormore_ap_mse = mean_squared_error(predict_oneormore_ap, test_oneormore_ap)
    oneormore_passing_mse = mean_squared_error(predict_oneormore_passing, test_oneormore_passing)
    
    return none_ap_mse, none_passing_mse, oneormore_ap_mse, oneormore_passing_mse

none_ap_mse, none_passing_mse, oneormore_ap_mse, oneormore_passing_mse = predictor()
print("MSE for Students Taking No AP exams in California in 2015: ", none_ap_mse)
print("MSE for Students Passing No AP exams in California in 2015: ", none_passing_mse)
print("MSE for Students Taking One or More AP exams in California in 2015: ", oneormore_ap_mse)
print("MSE for Students Passing One or More AP exams in California in 2015: ", oneormore_passing_mse)
    
    

MSE for Students Taking No AP exams in California in 2015:  1696.0434014657346
MSE for Students Passing No AP exams in California in 2015:  1816.2829910415514
MSE for Students Taking One or More AP exams in California in 2015:  12083.408802793369
MSE for Students Passing One or More AP exams in California in 2015:  5679.750827187407
